In [1]:
from utils.dataloader import DataLoader
from utils.osmGraph import GraphFromHmlFile, GraphFromPlace
import os
import io
import geopandas as gpd
from utils.apiKeys import Censusapi
import requests
import plotly.graph_objects as go
import numpy as np
import plotly
import osmnx as ox


In [2]:
dataFolder = "./data"

In [3]:
graphFileName = "minnesota.graphml"
#graphFileName = "minneapolis.graphml"
graph = GraphFromHmlFile(os.path.join(dataFolder, graphFileName))
print(len(graph.edgesGdf))

967835


In [4]:
stationFileName = "stationData_MN.csv"
stationData = DataLoader.readCSV(os.path.join(dataFolder, stationFileName))

In [5]:
stationData.describe()

,Unnamed: 0,ZIP,Latitude,Longitude,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count
count,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000
mean,279.000000,55482.341682,45.113029,-93.351857,0.039356,1.930233,0.431127
std,161.513673,405.437592,0.795041,0.799114,0.547287,1.322726,1.535190
min,0.000000,53662.000000,43.633850,-96.767240,0.000000,0.000000,0.000000
25%,139.500000,55124.500000,44.858374,-93.446979,0.000000,1.000000,0.000000
50%,279.000000,55405.000000,44.969550,-93.275995,0.000000,2.000000,0.000000
75%,418.500000,55606.000000,45.078028,-93.088530,0.000000,2.000000,0.000000
max,558.000000,56636.000000,48.054129,-89.692650,12.000000,16.000000,12.000000


In [6]:
stationDataSelected = stationData[stationData['ZIP'] == 55455]

In [7]:
stationDataSelected.index

Int64Index([36, 52, 76, 77, 80, 81, 105, 188], dtype='int64')

In [8]:
stationPositionList = graph.extractPositionListFrom(stationDataSelected)

In [9]:
stationPositionList

[(44.974036, -93.230989),
 (44.969677, -93.243328),
 (44.97145, -93.236027),
 (44.976321, -93.23395),
 (44.978214, -93.224562),
 (44.976277, -93.228946),
 (44.973875, -93.230153),
 (44.979167, -93.228946)]

In [10]:
censusFileName = "censusData_MN_Population.csv"
censusData = DataLoader.readCSV(os.path.join(dataFolder, censusFileName))

In [11]:
censusData.describe()

,Unnamed: 0,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,ALAND,AWATER,INTPTLAT,INTPTLON,B01001_001E,state,county,tract,block group,geoid
count,4706.000000,4706.0,4706.000000,4706.000000,4706.000000,4.706000e+03,4.706000e+03,4.706000e+03,4706.000000,4706.000000,4706.000000,4706.0,4706.000000,4706.000000,4706.000000,4.706000e+03
mean,2352.500000,27.0,80.320017,204739.733532,2.238207,2.708052e+11,4.382324e+07,4.026745e+06,45.232614,-93.594910,1190.005525,27.0,80.320017,204739.733532,2.238207,2.708052e+11
std,1358.649513,0.0,50.377466,307551.101353,1.143291,5.039194e+08,1.459062e+08,7.060843e+07,0.967383,1.000160,535.489228,0.0,50.377466,307551.101353,1.143291,5.039194e+08
min,0.000000,27.0,1.000000,100.000000,0.000000,2.700177e+11,0.000000e+00,0.000000e+00,43.511983,-97.029791,0.000000,27.0,1.000000,100.000000,0.000000,2.700177e+11
25%,1176.250000,27.0,41.000000,26708.750000,1.000000,2.704145e+11,6.362640e+05,0.000000e+00,44.819986,-93.830478,816.250000,27.0,41.000000,26708.750000,1.000000,2.704145e+11
50%,2352.500000,27.0,53.000000,60739.000000,2.000000,2.705313e+11,1.582888e+06,1.610050e+04,44.984657,-93.307253,1099.000000,27.0,53.000000,60739.000000,2.000000,2.705313e+11
75%,3528.750000,27.0,123.000000,130276.500000,3.000000,2.712304e+11,1.627754e+07,4.741442e+05,45.334613,-93.114646,1481.750000,27.0,123.000000,130276.500000,3.000000,2.712304e+11
max,4705.000000,27.0,173.000000,990100.000000,7.000000,2.717397e+11,3.877442e+09,4.185238e+09,48.973930,-90.016378,4484.000000,27.0,173.000000,990100.000000,7.000000,2.717397e+11


In [12]:
censusDataSelected = censusData[censusData['TRACTCE']==104901]

In [13]:
censusDataSelected

,Unnamed: 0,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,INTPTLAT,INTPTLON,geometry,NAME,B01001_001E,state,county,tract,block group,geoid
559,559,27,53,104901,1,270531049011,Block Group 1,G5030,S,421213,...,44.971982,-93.233441,"POLYGON ((-93.239268 44.973257, -93.2388 44.97...","Block Group 1, Census Tract 1049.01, Hennepin ...",4484,27,53,104901,1,270531049011
763,763,27,53,104901,2,270531049012,Block Group 2,G5030,S,908632,...,44.977829,-93.234230,"POLYGON ((-93.243853 44.979307999999996, -93.2...","Block Group 2, Census Tract 1049.01, Hennepin ...",1998,27,53,104901,2,270531049012


In [14]:
censusDataSelected

,Unnamed: 0,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,INTPTLAT,INTPTLON,geometry,NAME,B01001_001E,state,county,tract,block group,geoid
559,559,27,53,104901,1,270531049011,Block Group 1,G5030,S,421213,...,44.971982,-93.233441,"POLYGON ((-93.239268 44.973257, -93.2388 44.97...","Block Group 1, Census Tract 1049.01, Hennepin ...",4484,27,53,104901,1,270531049011
763,763,27,53,104901,2,270531049012,Block Group 2,G5030,S,908632,...,44.977829,-93.234230,"POLYGON ((-93.243853 44.979307999999996, -93.2...","Block Group 2, Census Tract 1049.01, Hennepin ...",1998,27,53,104901,2,270531049012


In [15]:
censusBlockList = graph.extractPositionListFrom(censusDataSelected, 'INTPTLAT', 'INTPTLON')

In [16]:
censusBlockList

[(44.9719823, -93.2334415), (44.9778291, -93.2342296)]

In [17]:
positionDict = {"station": stationPositionList, "census block group": censusBlockList}

In [18]:
figureFolder = "./results"
figureName = "test.html"
graph.plotPointList(positionDict, ['green', 'red'] ,os.path.join(figureFolder,figureName))

red


In [19]:
stationNodeList = graph.positionList2NodeList(stationPositionList)

In [20]:
censusNodeList = graph.positionList2NodeList(censusBlockList)

In [21]:
pathToStations, distanceToStations = graph.shortestPathForEachODPari(censusNodeList, stationNodeList)
distanceToStations

({246710361: [[246710361,
    246710356,
    1821777933,
    246710346,
    34198362,
    34665658,
    34646309,
    244113404,
    33359705,
    261298089,
    244111945,
    320363001],
   [246710361,
    246710356,
    1821777933,
    246710346,
    34198362,
    34656986,
    1813420189,
    33632334,
    1449264315,
    33304001,
    1449264307,
    244113498,
    34667306,
    34645936,
    34655777,
    33822058],
   [246710361, 246710356, 9421986672],
   [246710361,
    246710356,
    1821777933,
    246710346,
    34198362,
    34656986,
    1813420189,
    33632334,
    3611069368,
    33842546],
   [246710361,
    246710356,
    1821777933,
    246710346,
    34198362,
    34656986,
    1813420189,
    33632334,
    3611069368,
    33842546,
    244112865,
    34174403,
    34178512,
    34504631,
    34566808],
   [246710361,
    246710356,
    1821777933,
    246710346,
    34198362,
    34656986,
    1813420189,
    33632334,
    3611069368,
    33842546,
    244112865,


In [22]:
pathToStations[246710361][0]

[246710361,
 246710356,
 1821777933,
 246710346,
 34198362,
 34665658,
 34646309,
 244113404,
 33359705,
 261298089,
 244111945,
 320363001]

In [23]:
figureName = "testPath.html"
graph.plotTrajList([graph.nodeList2GPSTraj(pathToStations[246710361][0])], os.path.join(figureFolder,figureName))

In [24]:
figureName = "pointAndTraj.html"
graph.plotTrajAndPoint([graph.nodeList2GPSTraj(pathToStations[246710361][0])], positionDict, ['green', 'red'] ,os.path.join(figureFolder,figureName))

green
green
red
red


In [25]:
columbusZipList = [55025, 55011, 55092, 55304, 55448, 55449, 55434, 55014, 44038]
stationDataColumbus_rural = stationData[stationData['ZIP'].isin(columbusZipList)]

In [26]:
stationDataColumbus_rural

,Unnamed: 0,ZIP,Latitude,Longitude,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,Access Days Time,EV Connector Types
5,5,55434,45.183283,-93.237039,0.0,1.0,0.0,24 hours daily,J1772
14,14,55304,45.221680,-93.309692,0.0,2.0,0.0,8am-5pm daily; employee and guest use only,J1772
62,62,55449,45.158552,-93.224782,0.0,2.0,0.0,24 hours daily,J1772
109,109,55025,45.276783,-93.005870,0.0,0.0,2.0,24 hours daily,CHADEMO J1772COMBO
127,127,55449,45.161522,-93.218838,0.0,6.0,0.0,24 hours daily; for customer use only,J1772 TESLA


In [27]:
censusDataColumbus_rural = censusData[censusData['TRACTCE']==50210]

In [28]:
censusDataColumbus_rural

,Unnamed: 0,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,INTPTLAT,INTPTLON,geometry,NAME,B01001_001E,state,county,tract,block group,geoid
1109,1109,27,3,50210,2,270030502102,Block Group 2,G5030,S,39421972,...,45.254914,-93.107678,"POLYGON ((-93.142338 45.213539999999995, -93.1...","Block Group 2, Census Tract 502.10, Anoka Coun...",739,27,3,50210,2,270030502102
1110,1110,27,3,50210,3,270030502103,Block Group 3,G5030,S,28109152,...,45.232473,-93.040978,"POLYGON ((-93.09357299999999 45.211458, -93.09...","Block Group 3, Census Tract 502.10, Anoka Coun...",824,27,3,50210,3,270030502103
1111,1111,27,3,50210,4,270030502104,Block Group 4,G5030,S,9500017,...,45.274154,-93.061887,"POLYGON ((-93.080727 45.254736, -93.080715 45....","Block Group 4, Census Tract 502.10, Anoka Coun...",719,27,3,50210,4,270030502104
3138,3138,27,3,50210,1,270030502101,Block Group 1,G5030,S,39127851,...,45.311719,-93.093370,POLYGON ((-93.14278399999999 45.32636199999999...,"Block Group 1, Census Tract 502.10, Anoka Coun...",1827,27,3,50210,1,270030502101


In [29]:
censusBlockList = graph.extractPositionListFrom(censusDataColumbus_rural, 'INTPTLAT', 'INTPTLON')
censusNodeList = graph.positionList2NodeList(censusBlockList)
stationPositionList = graph.extractPositionListFrom(stationDataColumbus_rural)
stationNodeList = graph.positionList2NodeList(stationPositionList)

In [58]:
stationNodeList.append(186298215)

In [59]:
pathToStations, distanceToStations = graph.shortestPathForEachODPari(censusNodeList, stationNodeList)
distanceToStations

{186248813: [20943.685,
  27751.668000000012,
  21355.72300000001,
  10217.878999999999,
  20667.34900000001,
  2710.653,
  476.493],
 5603290650: [23299.662999999993,
  30107.646000000004,
  23711.700999999997,
  6883.620000000003,
  23023.326999999997,
  4668.557999999999,
  6902.717999999999],
 9569913070: [22397.770999999993,
  26696.554999999986,
  22809.809000000005,
  6249.419999999999,
  22121.435,
  3596.468,
  5584.351000000001],
 186253453: [22194.87999999999,
  23899.778999999984,
  23735.787,
  9951.639000000001,
  23047.413,
  7510.748000000002,
  8443.154000000002]}

In [32]:
pathToStations[186248813][3]

[186248813,
 186298215,
 186319452,
 4602470025,
 186282765,
 186273706,
 186273702,
 186273312,
 5483894275,
 5573568341,
 186273668,
 186269270,
 186315575,
 188506202,
 9972824659,
 9972824658,
 6203449773,
 188428456,
 1429394132,
 1429385444,
 1448711660]

In [33]:
figureFolder = "./results"
figureName = "columbus.html"
positionDict = {"station": stationPositionList, "census block group": censusBlockList}
graph.plotPointList(positionDict, ['green', 'red'] ,os.path.join(figureFolder,figureName))

red


In [35]:
graph.nodesGdf.head()

,y,x,street_count,highway,ref,geometry
osmid,,,,,,
158775549,43.500085,-92.692754,3,NaN,NaN,POINT (-92.69275 43.50008)
160498917,43.499865,-92.878523,3,NaN,NaN,POINT (-92.87852 43.49987)
160498975,43.499763,-92.979487,3,NaN,NaN,POINT (-92.97949 43.49976)
160499711,43.499920,-92.858607,3,NaN,NaN,POINT (-92.85861 43.49992)
160499721,43.499943,-92.828628,3,NaN,NaN,POINT (-92.82863 43.49994)


In [60]:
figureFolder = "./results"
figureName = "newStation.html"
newStationxs, newStationys = graph.nodesGdf.loc[186298215, 'geometry'].xy
positionDict = {"station": stationPositionList, "census block group": censusBlockList, "New station": [[newStationys[0], newStationxs[0]]]}
graph.plotPointList(positionDict, ['green', 'red', 'blue'] ,os.path.join(figureFolder,figureName))

red
blue
